<a href="https://colab.research.google.com/github/jansle2005/Data-Science-Repository/blob/master/Twitter_Scraping_OK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter Scrapping - 20/02/2022 - 01:40h

## Instalações de Bibliotecas

In [ ]:
!pip install snscrape
!pip install pandas

## Importa Bibliotecas

In [10]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import datetime
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

## Input dos Filtros

In [101]:
text = input('Digite o texto da consulta a ser correspondido (ou deixe em branco pressionando enter)')
# username = input('Digite o(s) nome(s) de usuário(s) específico(s) de uma conta do Twitter sem @ (ou deixe em branco pressionando enter): ')
data1 = input('Digite a data de início neste formato aaaa-mm-dd (ou deixe em branco pressionando enter): ')
data2 = input('Digite a data final neste formato yyyy-mm-dd (ou deixe em branco pressionando enter): ')
count = int(input('Digite o número máximo de tweets ou digite -1 para recuperar todos os tweets possíveis: '))
retweet ='s' #input('Excluir Retweets? (s/n): ')
replies ='s' #input('Excluir Replies? (s/n): ')


Digite o texto da consulta a ser correspondido (ou deixe em branco pressionando enter)Juscelino Filho
Digite a data de início neste formato aaaa-mm-dd (ou deixe em branco pressionando enter): 2022-02-01
Digite a data final neste formato yyyy-mm-dd (ou deixe em branco pressionando enter): 2022-02-14
Digite o número máximo de tweets ou digite -1 para recuperar todos os tweets possíveis: 1000


In [105]:
def day_after(data2):
  data = datetime.datetime.strptime(data2, '%Y-%m-%d')
  dia_seguinte = data + datetime.timedelta(days=1)
  data_formatada = dia_seguinte.strftime('%Y-%m-%d')
  return data_formatada

data2=day_after(data2)

In [106]:
def search(text,username,data1,data2,retweet,replies):
    global filename
    q = text+ ' lang:pt'
    print('q =',q)
    if username!='':
        q += f" from:{username}"   
    print('q =',q)
    if data1=='':
        data1 = datetime.datetime.strftime(datetime.datetime.strptime(data2, '%Y-%m-%d') - datetime.timedelta(days=7), '%Y-%m-%d')
    print('q =',q)
    q += f" since:{data1}" 
    print('q =',q)
    if data2=='':
        data2 = datetime.datetime.strftime(datetime.date.today(), '%Y-%m-%d')
    print('q =',q)
    q += f" until:{data2}"
    print('q =',q)
    if retweet == 's':
        q += f" exclude:retweets"
    print('q =',q)
    if replies == 's':
        q += f" exclude:replies"
    print('q =',q)
    if username!='' and text!='':
        filename = f"{data1}_{data2}_{username}_{text}.csv"
        
    elif username!="":
        filename = f"{data1}_{data2}_{username}.csv"
    
    else:
        filename = f"{data1}_{data2}_{text}.csv"
    
    print(filename)
    print('ÚLTIMO q =',q)
    return q

In [107]:
q = search(text,username,data1,data2,retweet,replies)

q = Juscelino Filho lang:pt
q = Juscelino Filho lang:pt
q = Juscelino Filho lang:pt
q = Juscelino Filho lang:pt since:2022-02-01
q = Juscelino Filho lang:pt since:2022-02-01
q = Juscelino Filho lang:pt since:2022-02-01 until:2022-02-16
q = Juscelino Filho lang:pt since:2022-02-01 until:2022-02-16 exclude:retweets
q = Juscelino Filho lang:pt since:2022-02-01 until:2022-02-16 exclude:retweets exclude:replies
2022-02-01_2022-02-16_Juscelino Filho.csv
ÚLTIMO q = Juscelino Filho lang:pt since:2022-02-01 until:2022-02-16 exclude:retweets exclude:replies


## Inicia o Scrapping

In [110]:
# Creating list to append tweet data 
tweets_list1 = []
tweets_df1=[]

# Using TwitterSearchScraper to scrape data and append tweets to list
if count == -1:
    for i,tweet in enumerate(tqdm_notebook(sntwitter.TwitterSearchScraper(q).get_items())):
        tweets_list1.append([tweet.date, 
                             tweet.id, 
                             tweet.rawContent, 
                             tweet.user.username,
                             tweet.hashtags,
                             tweet.replyCount,
                             tweet.retweetCount,
                             tweet.likeCount])
else:
    with tqdm_notebook(total=count) as pbar:
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()):
            if i>=count: #number of tweets you want to scrape
                break
            tweets_list1.append([tweet.date, 
                                 tweet.id, 
                                 tweet.rawContent, 
                                 tweet.user.username,
                                 tweet.hashtags,
                                 tweet.replyCount,
                                 tweet.retweetCount,
                                 tweet.likeCount])
            pbar.update(1)


# Creating a dataframe from the tweets list above 
tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username',
                                'Hashtags','ReplyCount','RetweetCount','LikeCount'])

  0%|          | 0/1000 [00:00<?, ?it/s]

In [111]:
tweets_df1.sort_values(by='DateTime')

,DateTime,TweetId,Text,Username,Hashtags,ReplyCount,RetweetCount,LikeCount
9,2022-02-02 15:56:54+00:00,1488904266010009602,"Na pessoa do presidente da @AL_Maranhao, @Othe...",DepJuscelino,[JuscelinoFilho],0,0,7
8,2022-02-02 15:57:48+00:00,1488904495937556490,Também quero parabenizar os desembargadores Pa...,DepJuscelino,"[JuscelinoFilho, TJMA]",0,0,0
7,2022-02-02 21:08:23+00:00,1488982655983095808,"Com o amigo e líder @efraimfilho, na abertura ...",DepJuscelino,"[JuscelinoFilho, PalavraETrabalho, CongressoNa...",0,0,3
6,2022-02-02 22:12:21+00:00,1488998750777397252,"Entrevista para a @BandTV, logo após a sessão ...",DepJuscelino,"[JuscelinoFilho, PalavraETrabalho, CongressoNa...",0,0,1
5,2022-02-08 22:48:38+00:00,1491182209969324033,O @TSEjusbr aprovou hoje a criação do nosso no...,DepJuscelino,[JuscelinoFilho],0,0,3
4,2022-02-09 16:03:36+00:00,1491442667901046786,"No Maranhão, a nova sigla será presidida pelo ...",marrapanarede,None,0,0,0
3,2022-02-10 12:14:19+00:00,1491747356504641536,Dois dias após o @TSEjusbr aprovar a criação d...,DepJuscelino,"[TBT, JuscelinoFilho, UniãoBrasil44]",1,0,3
2,2022-02-11 14:06:51+00:00,1492138063652347911,"Ao #JornalDaDifusora, falei sobre a criação do...",DepJuscelino,"[JornalDaDifusora, Maranhão, JuscelinoFilho, U...",1,0,1
1,2022-02-11 22:02:12+00:00,1492257689153548297,PL do veneno Como votaram deputados federais d...,publiconews,None,0,2,2
0,2022-02-15 20:21:58+00:00,1493682017845850120,"Lamento, com tristeza, a morte da vereadora Ro...",DepJuscelino,[JuscelinoFilho],0,0,0


## Fazer Análise de Sentimento

In [112]:
from textblob import TextBlob

# função para calcular o número de palavras em cada tweet
def contar_palavras(tweet):
    return len(tweet.split())

# função para calcular a polaridade do tweet usando TextBlob
def calcular_polaridade(tweet):
    tb = TextBlob(tweet)
    return tb.sentiment.polarity

# função para calcular o sentimento do tweet usando TextBlob
def calcular_sentimento(tweet):
    tb = TextBlob(tweet)
    return tb.sentiment.subjectivity



# adicionar coluna com o número de palavras de cada tweet
tweets_df1['num_palavras_Blob'] = tweets_df1['Text'].apply(contar_palavras)

# adicionar coluna com a polaridade de cada tweet
tweets_df1['polaridade_Blob'] = tweets_df1['Text'].apply(calcular_polaridade)

# adicionar coluna com o sentimento de cada tweet
tweets_df1['sentimento_Blob'] = tweets_df1['Text'].apply(calcular_sentimento)

In [114]:
display(tweets_df1)

,DateTime,TweetId,Text,Username,Hashtags,ReplyCount,RetweetCount,LikeCount,num_palavras_Blob,polaridade_Blob,sentimento_Blob
0,2022-02-15 20:21:58+00:00,1493682017845850120,"Lamento, com tristeza, a morte da vereadora Ro...",DepJuscelino,[JuscelinoFilho],0,0,0,39,0.0,0.0
1,2022-02-11 22:02:12+00:00,1492257689153548297,PL do veneno Como votaram deputados federais d...,publiconews,None,0,2,2,42,0.0,0.0
2,2022-02-11 14:06:51+00:00,1492138063652347911,"Ao #JornalDaDifusora, falei sobre a criação do...",DepJuscelino,"[JornalDaDifusora, Maranhão, JuscelinoFilho, U...",1,0,1,41,0.0,0.0
3,2022-02-10 12:14:19+00:00,1491747356504641536,Dois dias após o @TSEjusbr aprovar a criação d...,DepJuscelino,"[TBT, JuscelinoFilho, UniãoBrasil44]",1,0,3,45,0.0,0.0
4,2022-02-09 16:03:36+00:00,1491442667901046786,"No Maranhão, a nova sigla será presidida pelo ...",marrapanarede,None,0,0,0,15,0.0,0.0
5,2022-02-08 22:48:38+00:00,1491182209969324033,O @TSEjusbr aprovou hoje a criação do nosso no...,DepJuscelino,[JuscelinoFilho],0,0,3,41,0.0,0.0
6,2022-02-02 22:12:21+00:00,1488998750777397252,"Entrevista para a @BandTV, logo após a sessão ...",DepJuscelino,"[JuscelinoFilho, PalavraETrabalho, CongressoNa...",0,0,1,22,0.0,0.0
7,2022-02-02 21:08:23+00:00,1488982655983095808,"Com o amigo e líder @efraimfilho, na abertura ...",DepJuscelino,"[JuscelinoFilho, PalavraETrabalho, CongressoNa...",0,0,3,29,0.0,0.0
8,2022-02-02 15:57:48+00:00,1488904495937556490,Também quero parabenizar os desembargadores Pa...,DepJuscelino,"[JuscelinoFilho, TJMA]",0,0,0,36,0.0,0.0
9,2022-02-02 15:56:54+00:00,1488904266010009602,"Na pessoa do presidente da @AL_Maranhao, @Othe...",DepJuscelino,[JuscelinoFilho],0,0,7,42,0.0,0.0
